In [1]:
  from google.colab import drive
  drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
  import os
  os.chdir("/content/drive/MyDrive/SMU/Courses/ML/Traditional-Models-SPC/")

In [ ]:
# this is the main referring github repo
# my self made few amendments to better fit our project and requirements
# !git clone https://github.com/songyouwei/ABSA-PyTorch.git

In [3]:
# fix the problem: ImportError: cannot import name 'SAVE_STATE_WARNING' from 'torch.optim.lr_scheduler' (/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py)
!pip install torch==1.4.0

     |████████████████████████████████| 753.4 MB 6.7 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.


In [4]:
!pip install -r requirements.txt

     |████████████████████████████████| 1.3 MB 8.5 MB/s 
     |████████████████████████████████| 2.9 MB 42.8 MB/s 
     |████████████████████████████████| 895 kB 50.1 MB/s 
     |████████████████████████████████| 1.1 MB 48.4 MB/s 


In [5]:
import pandas as pd
import numpy as np

# generate raw file and print the loaded dataset

train_csv_filename = 'datasets/train_update.csv'
test_csv_filename = 'datasets/test_phase_1_update.csv'

train_raw_filename = 'datasets/train.raw'
test_raw_filename = 'datasets/test.raw'

result_filename = 'datasets/result.csv'

## Dataset loding and preprocessing

In [ ]:
df_train = pd.read_csv(train_csv_filename)
df_train

,ID,Claim,Evidence,Label
0,0,Antidepressants increase the severity of migra...,Tricyclics were also more likely to reduce the...,2
1,1,Citrullinated proteins externalized in neutrop...,These observations implicate accelerated NETos...,1
2,2,Recognition of start codons depends on the tra...,IF3 and tRNA undergo large conformational chan...,1
3,3,Ca2+ cycling is a UCP1-dependent thermogenic m...,"Conversely, enhanced Ca2+ cycling by activatio...",2
4,4,Weighed food records (WFR) result in poor comp...,The Hippo pathway controls organ size and tiss...,0
...,...,...,...,...
1174,1174,Birth-weight is negatively associated with bre...,The relative risk estimate for breast cancer c...,2
1175,1175,Rhythmic expression of Cry1 translates directl...,Cry1 expression was elevated during the night-...,1
1176,1176,Mice lacking Sirt1 in Sf1-expressing neurons h...,"Also, mutant mice have increased susceptibilit...",1
1177,1177,Microglia are an innate immune cell type of th...,"Here, we develop a conceptual framework for fu...",2


In [ ]:
df_train['Label'].value_counts()

1    602
2    316
0    261
Name: Label, dtype: int64

In [ ]:
df_test = pd.read_csv(test_csv_filename)
df_test

,ID,Claim,Evidence
0,0,Beta-band coherence is diminished for visible ...,This result suggests that traces of a heavy me...
1,1,Physical activity level is associated with the...,The best diagnostic performance is obtained wh...
2,2,Autologous transplantation of mesenchymal stem...,"After 6 months, 4 of 53 patients (7.5%) in the..."
3,3,RAD52 is involved in break-induced DNA replica...,"At CFSs, this fragility is associated with a d..."
4,4,Human embryonic stem cells give rise to cell t...,Embryonic stem cells have the ability to remai...
...,...,...,...
145,145,Origin gross domestic product(GDP) is positive...,"However, mature size DNA products accumulated ..."
146,146,Surgical treatment is not superior to non-surg...,RESULTS There was no significant mean treatmen...
147,147,Cancer-associated fibroblasts (CAFs) interact ...,"Further, we demonstrated that LXR is poly(ADP-..."
148,148,Persistor cells are one reason for incomplete ...,CONTEXT The epidemic of heart failure has yet ...


In [ ]:
df_test['Label'] = 0
df_test

,ID,Claim,Evidence,Label
0,0,Beta-band coherence is diminished for visible ...,This result suggests that traces of a heavy me...,0
1,1,Physical activity level is associated with the...,The best diagnostic performance is obtained wh...,0
2,2,Autologous transplantation of mesenchymal stem...,"After 6 months, 4 of 53 patients (7.5%) in the...",0
3,3,RAD52 is involved in break-induced DNA replica...,"At CFSs, this fragility is associated with a d...",0
4,4,Human embryonic stem cells give rise to cell t...,Embryonic stem cells have the ability to remai...,0
...,...,...,...,...
145,145,Origin gross domestic product(GDP) is positive...,"However, mature size DNA products accumulated ...",0
146,146,Surgical treatment is not superior to non-surg...,RESULTS There was no significant mean treatmen...,0
147,147,Cancer-associated fibroblasts (CAFs) interact ...,"Further, we demonstrated that LXR is poly(ADP-...",0
148,148,Persistor cells are one reason for incomplete ...,CONTEXT The epidemic of heart failure has yet ...,0


In [ ]:
# generate raw file for training set
df = pd.DataFrame(df_train, columns=['Claim', 'Evidence', 'Label'])
# preprocess the textual column remove \n
df['Claim'] = df['Claim'].replace('\n', ' ', regex = True)
df['Evidence'] = df['Evidence'].replace('\n', ' ', regex = True)
df = df.dropna()
print(df)

# use the csv file to generate the raw file that used for the model running
# save .raw file
df.to_csv(train_raw_filename, sep='\n', index = False, header = False)
print('the generate for ['+train_raw_filename+'] files is completed')

                                                  Claim  \
0     Antidepressants increase the severity of migra...   
1     Citrullinated proteins externalized in neutrop...   
2     Recognition of start codons depends on the tra...   
3     Ca2+ cycling is a UCP1-dependent thermogenic m...   
4     Weighed food records (WFR) result in poor comp...   
...                                                 ...   
1174  Birth-weight is negatively associated with bre...   
1175  Rhythmic expression of Cry1 translates directl...   
1176  Mice lacking Sirt1 in Sf1-expressing neurons h...   
1177  Microglia are an innate immune cell type of th...   
1178  There is no increased risk of hypospadias with...   

                                               Evidence  Label  
0     Tricyclics were also more likely to reduce the...      2  
1     These observations implicate accelerated NETos...      1  
2     IF3 and tRNA undergo large conformational chan...      1  
3     Conversely, enhanced Ca2+

In [ ]:
# generate raw file for testing set
df = pd.DataFrame(df_test, columns=['Claim', 'Evidence', 'Label'])
# preprocess the textual column remove \n
df['Claim'] = df['Claim'].replace('\n', ' ', regex = True)
df['Evidence'] = df['Evidence'].replace('\n', ' ', regex = True)
df = df.dropna()
print(df)

# use the csv file to generate the raw file that used for the model running
# save .raw file
df.to_csv(test_raw_filename, sep='\n', index = False, header = False)
print('the generate for ['+test_raw_filename+'] files is completed')

                                                 Claim  \
0    Beta-band coherence is diminished for visible ...   
1    Physical activity level is associated with the...   
2    Autologous transplantation of mesenchymal stem...   
3    RAD52 is involved in break-induced DNA replica...   
4    Human embryonic stem cells give rise to cell t...   
..                                                 ...   
145  Origin gross domestic product(GDP) is positive...   
146  Surgical treatment is not superior to non-surg...   
147  Cancer-associated fibroblasts (CAFs) interact ...   
148  Persistor cells are one reason for incomplete ...   
149  Normal expression of RUNX1 causes tumorsupress...   

                                              Evidence  Label  
0    This result suggests that traces of a heavy me...      0  
1    The best diagnostic performance is obtained wh...      0  
2    After 6 months, 4 of 53 patients (7.5%) in the...      0  
3    At CFSs, this fragility is associated with

## Run the codes

In [ ]:
!python dependency_graph.py

In [6]:
!python train.py --model_name lstm --dataset phase-1 --num_epoch 10 --device cuda:0 --polarities_dim 3  --valset_ratio 0.10

loading tokenizer: phase-1_tokenizer.dat
loading embedding_matrix: 300_phase-1_embedding_matrix.dat
cuda memory allocated: 11683328
> n_trainable_params: 723303, n_nontrainable_params: 2197200
> training arguments:
>>> model_name: lstm
>>> dataset: phase-1
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7fc5b2540290>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 10
>>> batch_size: 16
>>> log_step: 10
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 85
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 5
>>> device: cuda:0
>>> seed: 1234
>>> valset_ratio: 0.1
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.lstm.LSTM'>
>>> dataset_file: {'train': './datasets/train.raw', 'test': './datasets/test.raw'}
>>> inputs_cols: ['text_indices']
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epoc

In [ ]:
!python train.py --model_name atae_lstm --dataset phase-1 --num_epoch 10 --device cuda:0 --polarities_dim 3  --valset_ratio 0.10

loading tokenizer: phase-1_tokenizer.dat
loading embedding_matrix: 300_phase-1_embedding_matrix.dat
cuda memory allocated: 19546624
> n_trainable_params: 2525703, n_nontrainable_params: 2197200
> training arguments:
>>> model_name: atae_lstm
>>> dataset: phase-1
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f4e340d63b0>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 10
>>> batch_size: 16
>>> log_step: 10
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 85
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 5
>>> device: cuda:0
>>> seed: 1234
>>> valset_ratio: 0.1
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.atae_lstm.ATAE_LSTM'>
>>> dataset_file: {'train': './datasets/train.raw', 'test': './datasets/test.raw'}
>>> inputs_cols: ['text_indices', 'aspect_indices']
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [ ]:
!python train.py --model_name ian --dataset phase-1 --num_epoch 10 --device cuda:0 --polarities_dim 3  --valset_ratio 0.10

loading tokenizer: phase-1_tokenizer.dat
loading embedding_matrix: 300_phase-1_embedding_matrix.dat
cuda memory allocated: 17469440
> n_trainable_params: 2168403, n_nontrainable_params: 2197200
> training arguments:
>>> model_name: ian
>>> dataset: phase-1
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f16d450d3b0>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 10
>>> batch_size: 16
>>> log_step: 10
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 85
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 5
>>> device: cuda:0
>>> seed: 1234
>>> valset_ratio: 0.1
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.ian.IAN'>
>>> dataset_file: {'train': './datasets/train.raw', 'test': './datasets/test.raw'}
>>> inputs_cols: ['text_indices', 'aspect_indices']
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [ ]:
!python train.py --model_name bert_spc --dataset phase-1 --num_epoch 10 --device cuda:0 --polarities_dim 3  --valset_ratio 0.10

Downloading: 100% 232k/232k [00:00<00:00, 641kB/s]
Downloading: 100% 570/570 [00:00<00:00, 573kB/s]
Downloading: 100% 440M/440M [00:07<00:00, 62.9MB/s]
cuda memory allocated: 439075328
> n_trainable_params: 109484547, n_nontrainable_params: 0
> training arguments:
>>> model_name: bert_spc
>>> dataset: phase-1
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f419c3783b0>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 10
>>> batch_size: 16
>>> log_step: 10
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 85
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 5
>>> device: cuda:0
>>> seed: 1234
>>> valset_ratio: 0.1
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.bert_spc.BERT_SPC'>
>>> dataset_file: {'train': './datasets/train.raw', 'test': './datasets/test.raw'}
>>> inputs_cols: ['concat_bert_indices', 'concat_segments_indices']
>>>>>>>>>

In [ ]:
!python train.py --model_name aen_bert --dataset phase-1 --num_epoch 10 --device cuda:0 --polarities_dim 3  --valset_ratio 0.10

cuda memory allocated: 452894208
> n_trainable_params: 112937661, n_nontrainable_params: 0
> training arguments:
>>> model_name: aen_bert
>>> dataset: phase-1
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7fc66d7013b0>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 10
>>> batch_size: 16
>>> log_step: 10
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 85
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 5
>>> device: cuda:0
>>> seed: 1234
>>> valset_ratio: 0.1
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.aen.AEN_BERT'>
>>> dataset_file: {'train': './datasets/train.raw', 'test': './datasets/test.raw'}
>>> inputs_cols: ['text_bert_indices', 'aspect_bert_indices']
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epoch: 0
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:

In [ ]:
!python train.py --model_name lcf_bert --dataset phase-1 --num_epoch 10 --device cuda:0 --polarities_dim 3  --valset_ratio 0.10

cuda memory allocated: 455608832
> n_trainable_params: 113617923, n_nontrainable_params: 0
> training arguments:
>>> model_name: lcf_bert
>>> dataset: phase-1
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f6ae234c3b0>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 10
>>> batch_size: 16
>>> log_step: 10
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 85
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 5
>>> device: cuda:0
>>> seed: 1234
>>> valset_ratio: 0.1
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.lcf_bert.LCF_BERT'>
>>> dataset_file: {'train': './datasets/train.raw', 'test': './datasets/test.raw'}
>>> inputs_cols: ['concat_bert_indices', 'concat_segments_indices', 'text_bert_indices', 'aspect_bert_indices']
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epoch: 0
loss: 1

In [ ]:
!python train.py --model_name memnet --dataset phase-1 --num_epoch 10 --device cuda:0 --polarities_dim 3  --valset_ratio 0.10

loading tokenizer: phase-1_tokenizer.dat
loading embedding_matrix: 300_phase-1_embedding_matrix.dat
cuda memory allocated: 10244096
> n_trainable_params: 362703, n_nontrainable_params: 2197200
> training arguments:
>>> model_name: memnet
>>> dataset: phase-1
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7efbeca103b0>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 10
>>> batch_size: 16
>>> log_step: 10
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 85
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 5
>>> device: cuda:0
>>> seed: 1234
>>> valset_ratio: 0.1
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.memnet.MemNet'>
>>> dataset_file: {'train': './datasets/train.raw', 'test': './datasets/test.raw'}
>>> inputs_cols: ['context_indices', 'aspect_indices']
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

## More about BERT

In [ ]:
!python train.py --model_name bert_spc --dataset phase-1 --num_epoch 10 --device cuda:0 --polarities_dim 3  --valset_ratio 0.10

### val ratios

In [ ]:
!python train.py --model_name bert_spc --dataset phase-1 --num_epoch 10 --device cuda:0 --polarities_dim 3  --valset_ratio 0.15

cuda memory allocated: 439075328
> n_trainable_params: 109484547, n_nontrainable_params: 0
> training arguments:
>>> model_name: bert_spc
>>> dataset: phase-1
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7fb6872733b0>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 10
>>> batch_size: 16
>>> log_step: 10
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 85
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 5
>>> device: cuda:0
>>> seed: 1234
>>> valset_ratio: 0.15
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.bert_spc.BERT_SPC'>
>>> dataset_file: {'train': './datasets/train.raw', 'test': './datasets/test.raw'}
>>> inputs_cols: ['concat_bert_indices', 'concat_segments_indices']
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epoch: 0
loss: 1.2435, acc: 0.4500
loss: 1.2322, acc: 0.428

In [ ]:
!python train.py --model_name bert_spc --dataset phase-1 --num_epoch 10 --device cuda:0 --polarities_dim 3  --valset_ratio 0.20

cuda memory allocated: 439075328
> n_trainable_params: 109484547, n_nontrainable_params: 0
> training arguments:
>>> model_name: bert_spc
>>> dataset: phase-1
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f9e9ef1e3b0>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 10
>>> batch_size: 16
>>> log_step: 10
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 85
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 5
>>> device: cuda:0
>>> seed: 1234
>>> valset_ratio: 0.2
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.bert_spc.BERT_SPC'>
>>> dataset_file: {'train': './datasets/train.raw', 'test': './datasets/test.raw'}
>>> inputs_cols: ['concat_bert_indices', 'concat_segments_indices']
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epoch: 0
loss: 1.2788, acc: 0.4062
loss: 1.1534, acc: 0.4437

In [ ]:
!python train.py --model_name bert_spc --dataset phase-1 --num_epoch 10 --device cuda:0 --polarities_dim 3  --valset_ratio 0.17

cuda memory allocated: 439075328
> n_trainable_params: 109484547, n_nontrainable_params: 0
> training arguments:
>>> model_name: bert_spc
>>> dataset: phase-1
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f4ccc6183b0>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 10
>>> batch_size: 16
>>> log_step: 10
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 85
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 5
>>> device: cuda:0
>>> seed: 1234
>>> valset_ratio: 0.17
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.bert_spc.BERT_SPC'>
>>> dataset_file: {'train': './datasets/train.raw', 'test': './datasets/test.raw'}
>>> inputs_cols: ['concat_bert_indices', 'concat_segments_indices']
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epoch: 0
loss: 1.2576, acc: 0.3750
loss: 1.1557, acc: 0.437

### epochs

In [ ]:
!python train.py --model_name bert_spc --dataset phase-1 --num_epoch 20 --device cuda:0 --polarities_dim 3  --valset_ratio 0.10

cuda memory allocated: 439075328
> n_trainable_params: 109484547, n_nontrainable_params: 0
> training arguments:
>>> model_name: bert_spc
>>> dataset: phase-1
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7fbcfcb8d3b0>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 20
>>> batch_size: 16
>>> log_step: 10
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 768
>>> pretrained_bert_name: bert-base-uncased
>>> max_seq_len: 85
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 5
>>> device: cuda:0
>>> seed: 1234
>>> valset_ratio: 0.1
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.bert_spc.BERT_SPC'>
>>> dataset_file: {'train': './datasets/train.raw', 'test': './datasets/test.raw'}
>>> inputs_cols: ['concat_bert_indices', 'concat_segments_indices']
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epoch: 0
loss: 1.1228, acc: 0.5437
loss: 1.0996, acc: 0.5312

### use the large bert pretrained model

In [ ]:
!python train.py --model_name bert_spc --dataset phase-1 --num_epoch 10 --device cuda:0 --polarities_dim 3  --valset_ratio 0.15 --pretrained_bert_name bert-large-uncased --bert_dim 1024

cuda memory allocated: 1341395456
> n_trainable_params: 335144963, n_nontrainable_params: 0
> training arguments:
>>> model_name: bert_spc
>>> dataset: phase-1
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7fbdaabf9440>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 10
>>> batch_size: 16
>>> log_step: 10
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 1024
>>> pretrained_bert_name: bert-large-uncased
>>> max_seq_len: 85
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 5
>>> device: cuda:0
>>> seed: 1234
>>> valset_ratio: 0.15
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.bert_spc.BERT_SPC'>
>>> dataset_file: {'train': './datasets/train.raw', 'test': './datasets/test.raw'}
>>> inputs_cols: ['concat_bert_indices', 'concat_segments_indices']
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epoch: 0
loss: 1.0761, acc: 0.5188
loss: 1.0236, acc: 0.

In [ ]:
!python train.py --model_name bert_spc --dataset phase-1 --num_epoch 20 --device cuda:0 --polarities_dim 3  --valset_ratio 0.15 --pretrained_bert_name bert-large-uncased --bert_dim 1024

cuda memory allocated: 1341395456
> n_trainable_params: 335144963, n_nontrainable_params: 0
> training arguments:
>>> model_name: bert_spc
>>> dataset: phase-1
>>> optimizer: <class 'torch.optim.adam.Adam'>
>>> initializer: <function xavier_uniform_ at 0x7f945cd65440>
>>> lr: 2e-05
>>> dropout: 0.1
>>> l2reg: 0.01
>>> num_epoch: 20
>>> batch_size: 16
>>> log_step: 10
>>> embed_dim: 300
>>> hidden_dim: 300
>>> bert_dim: 1024
>>> pretrained_bert_name: bert-large-uncased
>>> max_seq_len: 85
>>> polarities_dim: 3
>>> hops: 3
>>> patience: 5
>>> device: cuda:0
>>> seed: 1234
>>> valset_ratio: 0.15
>>> local_context_focus: cdm
>>> SRD: 3
>>> model_class: <class 'models.bert_spc.BERT_SPC'>
>>> dataset_file: {'train': './datasets/train.raw', 'test': './datasets/test.raw'}
>>> inputs_cols: ['concat_bert_indices', 'concat_segments_indices']
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
epoch: 0
loss: 1.0761, acc: 0.5188
loss: 1.0236, acc: 0.